In [30]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from io import StringIO 
from tqdm.notebook import tqdm

import datetime
import time
from datetime import datetime, timedelta

In [3]:
# Make the list of dates

start_date = datetime(2015, 1, 1)
end_date = datetime.now()

date_list = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d')
             for i in range((end_date - start_date).days + 1)]

In [38]:
base_url = 'https://www.wunderground.com/history/daily/il/haifa/LLHA/date/'

driver = webdriver.Chrome()

for date in tqdm(date_list, desc="Scraping Pages"):
    try:
        url = f"{base_url}{date}"
        driver.get(url)
        tables = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "observation-table")))
        for table in tables:
            newTable = pd.read_html(StringIO(table.get_attribute('outerHTML')))
            if newTable:
                newTable[0].to_csv(f"saved_tables//{date}.csv")
        time.sleep(1)
    except TimeoutException:
        pass

Scraping Pages:   0%|          | 0/2639 [00:00<?, ?it/s]

In [31]:
url

'https://www.wunderground.com/history/daily/il/haifa/LLHA/date/2017-09-30'

In [37]:
date_list[1003]

'2017-09-30'